In [ ]:
# ==== 전맥락 + 화자태깅 학습 스크립트 ====
import os, json, random, numpy as np, torch, re, glob
from typing import List
import pandas as pd
from datasets import Dataset as HFDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback
)

In [ ]:
# (1) 혹시 모를 잔여 마운트 해제
!fusermount -u /content/drive 2>/dev/null || true
!umount /content/drive 2>/dev/null || true

# (2) 로컬에 남아있는 가짜 /content/drive 폴더/파일 제거 후 재생성
!rm -rf /content/drive
import os
os.makedirs("/content/drive", exist_ok=True)

# (3) 강제 재마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# (4) 확인
import os
print("is mount?", os.path.ismount("/content/drive"))
!ls -la /content/drive | head -n 20
!ls -la /content/drive/MyDrive | head -n 20

KeyboardInterrupt: 

In [ ]:
# -------------------- 0) 공통 설정 --------------------
os.environ["HF_ALLOW_CODE_EXECUTION"] = "1"
os.environ["TRANSFORMERS_ALLOW_CODE_EXECUTION"] = "1"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# *** 필요한 경로만 바꿔 ***
DATA_PATH = "/content/drive/MyDrive/metrics/혐오조롱표현탐지용데이터 15400개.jsonl"
BASE_DIR  = "/content/drive/MyDrive/model_backup/results_ctx_tgt_best"
os.makedirs(BASE_DIR, exist_ok=True)


Device: cuda


In [ ]:
print("after mount -> drive exists?", os.path.isdir("/content/drive"))
print("MyDrive exists?", os.path.isdir("/content/drive/MyDrive"))

after mount -> drive exists? True
MyDrive exists? True


In [ ]:
def load_jsonl(path: str):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for lineno, line in enumerate(f, start=1):
            s = line.strip()
            if not s: continue
            try:
                rows.append(json.loads(s))
            except json.JSONDecodeError as e:
                print(f"[WARN] JSON decode error at line {lineno}: {e}")
    return rows

def map_label(lbl: str) -> int:
    if not lbl: return 0
    return 1 if str(lbl).strip().lower() == "toxic" else 0

def ensure_speaker_prefixes(utts: List[str]) -> List[str]:
    out = []
    for i, u in enumerate(utts):
        s = u.strip()
        if s.startswith("A:") or s.startswith("B:"):
            out.append(s)
        else:
            # 규칙이 없다면 A/B 번갈아 태깅 (데이터 포맷에 맞게 수정 가능)
            out.append(("A: " if i % 2 == 0 else "B: ") + s)
    return out

def get_speaker(line: str) -> str:
    line = line.strip()
    if line.startswith("A:"): return "A"
    if line.startswith("B:"): return "B"
    return "U"

def build_input(utterances: List[str], k: int) -> str:
    """
    전맥락 [CTX] ... [/CTX] + 타깃 화자 [TGT_SPK=X] ... [/TGT]
    """
    utts = ensure_speaker_prefixes(utterances)
    ctx = utts[:k]
    tgt = utts[k]
    tgt_spk = get_speaker(tgt)
    parts = []
    if ctx:
        parts.append("[CTX]")
        parts.extend(ctx)
        parts.append("[/CTX]")
    parts.append(f"[TGT_SPK={tgt_spk}] {tgt} [/TGT]")
    return "\n".join(parts)

raw = load_jsonl(DATA_PATH)
print(f"✅ 데이터 로드: {len(raw)} rows")

records = []
bad = 0
for r in raw:
    utts = r.get("utterances") or r.get("context") or []
    idx = r.get("target_index", None)
    lbl = r.get("label", None)
    if not isinstance(utts, list) or idx is None or idx < 0 or idx >= len(utts):
        bad += 1
        continue
    records.append({
        "dialogue_id": r.get("dialogue_id", r.get("id", "")),
        "text": build_input(utts, idx),
        "label": map_label(lbl)
    })
if bad:
    print(f"⚠️ 무시된 레코드: {bad}")

df = pd.DataFrame(records)
print(df.head(2))
print("라벨 분포:\n", df["label"].value_counts())

def load_tok_and_model():
    try:
        tok = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
        mdl = AutoModelForSequenceClassification.from_pretrained(
            "monologg/kobert", trust_remote_code=True, num_labels=2
        )
        print("✅ KoBERT 로드")
    except Exception as e:
        print("❌ KoBERT 실패 → mBERT로 대체:", e)
        tok = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
        mdl = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-multilingual-cased", num_labels=2
        )
        print("✅ mBERT 로드")
    return tok, mdl

tokenizer, _ = load_tok_and_model()
orig_save_vocab = getattr(tokenizer, "save_vocabulary", None)
if callable(orig_save_vocab):
    def _patched_save_vocabulary(save_directory, *args, **kwargs):
        if "filename_prefix" in kwargs: kwargs.pop("filename_prefix")
        return orig_save_vocab(save_directory, *args, **kwargs)
    tokenizer.save_vocabulary = _patched_save_vocabulary
# 스페셜 토큰 등록 (권장)
SPECIAL_TOKENS = {
    "additional_special_tokens": ["[CTX]", "[/CTX]", "[TGT_SPK=A]", "[TGT_SPK=B]", "[TGT_SPK=U]", "[TGT]", "[/TGT]"]
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

def model_init():
    # KoBERT 우선, 실패 시 mBERT 대체 (원래 함수 재사용)
    _, mdl = load_tok_and_model()
    # 스페셜 토큰 반영
    mdl.resize_token_embeddings(len(tokenizer))
    # 레이블 매핑 유지
    mdl.config.label2id = {"non_toxic": 0, "toxic": 1}
    mdl.config.id2label = {0: "non_toxic", 1: "toxic"}
    return mdl


# -------------------- 5) 메트릭/트레이너 --------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    out = {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
        "f1": f1_score(labels, preds, zero_division=0),
    }
    try:
        prob_pos = (np.exp(logits) / np.exp(logits).sum(-1, keepdims=True))[:, 1]
        out["roc_auc"] = roc_auc_score(labels, prob_pos)
    except Exception:
        out["roc_auc"] = float("nan")
    return out


# -------------------- 4) 가중치 & Focal Loss --------------------
pos = int((train_df["label"] == 1).sum())
neg = int((train_df["label"] == 0).sum())
ratio = neg / max(1, pos)
w_pos = float(min(3.0, max(1.0, ratio)))  # 1~3로 clamp (과탐 방지)
class_weights = torch.tensor([1.0, w_pos], dtype=torch.float32).to(device)

import torch.nn.functional as F
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=1.5, reduction="mean"):
        super().__init__()
        self.alpha = alpha  # class_weights 텐서
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, logits, target):
        ce = F.cross_entropy(logits, target, weight=self.alpha, reduction="none")
        pt = torch.softmax(logits, dim=-1).gather(1, target.view(-1,1)).squeeze(1).clamp_(1e-6, 1-1e-6)
        loss = ((1-pt) ** self.gamma) * ce
        return loss.mean() if self.reduction=="mean" else loss.sum()

focal = FocalLoss(alpha=class_weights, gamma=1.5)

def custom_compute_loss(model, inputs, return_outputs=False, **kwargs):
    labels = inputs.get("labels")
    outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
    logits = outputs.logits
    loss = focal(logits.view(-1, 2), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

# -------------------- 3) Dataset/Tokenize --------------------
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding=False)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train_ds = HFDataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = HFDataset.from_pandas(val_df.reset_index(drop=True))

remove_cols = [c for c in ["text", "dialogue_id", "__index_level_0__"] if c in train_ds.column_names]
train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=remove_cols)
val_ds   = val_ds.map(tokenize_fn,   batched=True, remove_columns=remove_cols)
train_ds = train_ds.rename_column("label", "labels")
val_ds   = val_ds.rename_column("label", "labels")
train_ds.set_format(type="torch")
val_ds.set_format(type="torch")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def hp_space_optuna(trial):
    return {
        # loguniform: 1e-5 ~ 5e-5 범위에서 학습률 탐색
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        # 배치·가적은 데이터/메모리 균형용
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16]),
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [16, 32]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4, 8]),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.12),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.05, 0.30),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type",
                              ["linear", "cosine", "constant_with_warmup", "polynomial"]),
        "max_grad_norm": trial.suggest_float("max_grad_norm", 0.5, 1.0),
        # 에폭도 가볍게 탐색 (과적합 방지 위해 3~5)
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 5),
    }

def compute_objective(metrics):
    # f1 기준 최대화
    return metrics.get("eval_f1", 0.0)

# ==================== [HPO] Trainer (탐색용) ====================
# 주의: HPO에서는 model 대신 model_init을 전달해야 함
hpo_args = TrainingArguments(
    output_dir=os.path.join(BASE_DIR, "hpo"),
    logging_dir=os.path.join(BASE_DIR, "hpo_logs"),
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=False,     # 탐색에서 best 로드는 비활성화
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False,
    report_to=[]
)

hpo_trainer = Trainer(
    model_init=model_init,
    args=hpo_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)
# focal loss 적용 (기존과 동일)
hpo_trainer.compute_loss = custom_compute_loss

print("🔎 HPO 시작 (Optuna)")
best_run = hpo_trainer.hyperparameter_search(
    hp_space=hp_space_optuna,
    direction="maximize",
    compute_objective=compute_objective,
    n_trials=20,          # 필요에 따라 늘리기(시간↑)
    backend="optuna",     # 기본 optuna
)
print("🏁 HPO 완료:", best_run)

# 결과 저장
with open(os.path.join(BASE_DIR, "best_params.json"), "w", encoding="utf-8") as f:
    json.dump(best_run.hyperparameters, f, ensure_ascii=False, indent=2)

BEST_PARAMS.update({
    "learning_rate": best_run.hyperparameters["learning_rate"],
    "per_device_train_batch_size": best_run.hyperparameters["per_device_train_batch_size"],
    "per_device_eval_batch_size": best_run.hyperparameters["per_device_eval_batch_size"],
    "gradient_accumulation_steps": best_run.hyperparameters["gradient_accumulation_steps"],
    "weight_decay": best_run.hyperparameters["weight_decay"],
    "warmup_ratio": best_run.hyperparameters["warmup_ratio"],
    "lr_scheduler_type": best_run.hyperparameters["lr_scheduler_type"],
})
NUM_TRAIN_EPOCHS = int(best_run.hyperparameters.get("num_train_epochs", NUM_TRAIN_EPOCHS))









✅ 데이터 로드: 15211 rows
  dialogue_id                                               text  label
0         001  [CTX]\nA: 부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네\nB: 맞아. ...      1
1         002  [CTX]\nA: 인방 보는 남자는 거르는게 맞다\nB: 특히 벗방보는 애들은 진짜...      0
라벨 분포:
 label
0    7957
1    7254
Name: count, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ KoBERT 로드


Map:   0%|          | 0/12168 [00:00<?, ? examples/s]

Map:   0%|          | 0/3043 [00:00<?, ? examples/s]

...

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.052100,0.098292,0.940519,0.974589,0.898691,0.935102,0.991830
2,0.035800,0.077037,0.948735,0.974359,0.916609,0.944602,0.992981


[I 2025-08-14 03:07:33,561] Trial 19 pruned. 


🏁 HPO 완료: BestRun(run_id='10', objective=0.9684575389948007, hyperparameters={'learning_rate': 3.3488606479071704e-05, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 16, 'gradient_accumulation_steps': 1, 'weight_decay': 0.04475501618361846, 'warmup_ratio': 0.16345827033905022, 'lr_scheduler_type': 'linear', 'max_grad_norm': 0.9850833568331282, 'num_train_epochs': 3}, run_summary=None)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/model_backup/results_ctx_tgt_best/best_params.json'

In [ ]:
BEST_PARAMS = {
    "learning_rate": 3.3488606479071704e-05,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 16,
    "gradient_accumulation_steps": 1,
    "weight_decay": 0.04475501618361846,
    "warmup_ratio": 0.16345827033905022,
    "lr_scheduler_type": "linear",
    "max_grad_norm": 0.9850833568331282
}

NUM_TRAIN_EPOCHS = 3  # HPO 결과 반영

final_args = TrainingArguments(
    output_dir=BASE_DIR,
    logging_dir=os.path.join(BASE_DIR, "logs"),
    eval_strategy="epoch",  # eval_strategy → evaluation_strategy (정식 파라미터명)
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    **BEST_PARAMS,
    report_to=[]
)

final_trainer = Trainer(
    model_init=model_init,  # 최종 학습도 동일 초기화 경로 사용
    args=final_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)
final_trainer.compute_loss = custom_compute_loss

print("🚀 최종 학습 시작 (HPO 최적 파라미터 적용)")
final_trainer.train()
print("✅ 최종 학습 종료")

import numpy as np
def find_best_threshold(trainer, val_ds, target_metric="f1"):
    preds = trainer.predict(val_ds)
    logits = preds.predictions
    labels = preds.label_ids
    prob_toxic = (np.exp(logits) / np.exp(logits).sum(-1, keepdims=True))[:, 1]
    best_t, best = 0.5, -1
    for t in np.linspace(0.05, 0.95, 19):
        yhat = (prob_toxic >= t).astype(int)
        m = {
            "accuracy": accuracy_score(labels, yhat),
            "precision": precision_score(labels, yhat, zero_division=0),
            "recall": recall_score(labels, yhat, zero_division=0),
            "f1": f1_score(labels, yhat, zero_division=0),
        }[target_metric]
        if m > best:
            best, best_t = m, t
    return float(best_t)

best_t = find_best_threshold(final_trainer, val_ds, target_metric="f1")
os.makedirs(BASE_DIR, exist_ok=True)
with open(os.path.join(BASE_DIR, "threshold.json"), "w", encoding="utf-8") as f:
    json.dump({"threshold": best_t}, f, ensure_ascii=False, indent=2)
print("✅ 최적 threshold 저장:", best_t)


final_trainer.save_model(BASE_DIR)           # 모델 저장
tokenizer.save_pretrained(BASE_DIR)          # 토크나이저 저장
print("✅ 모델과 토크나이저 저장 완료:", BASE_DIR)

/tmp/ipython-input-2513236155.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ KoBERT 로드
🚀 최종 학습 시작 (HPO 최적 파라미터 적용)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ KoBERT 로드


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.068500,0.148934,0.922445,0.984064,0.851137,0.912786,0.992133
2,0.041900,0.045795,0.974039,0.964141,0.982081,0.973028,0.994966
3,0.014100,0.062914,0.969767,0.974843,0.961406,0.968078,0.995384


✅ 최종 학습 종료


✅ 최적 threshold 저장: 0.44999999999999996
✅ 모델과 토크나이저 저장 완료: /content/drive/MyDrive/model_backup/results_ctx_tgt_best


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==== 전맥락 + 화자태깅 추론 스크립트 ====
import os, json, torch, re
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# (학습과 동일 경로)
BASE_DIR = "/content/drive/MyDrive/model_backup/results_ctx_tgt_best"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def load_threshold(model_dir, default=0.1):
    p = os.path.join(model_dir, "threshold.json")
    if os.path.exists(p):
        try:
            return float(json.load(open(p, "r", encoding="utf-8"))["threshold"])
        except Exception:
            pass
    return default

def ensure_speaker_prefixes(utts):
    out = []
    for i, u in enumerate(utts):
        s = u.strip()
        if s.startswith("A:") or s.startswith("B:"):
            out.append(s)
        else:
            out.append(("A: " if i % 2 == 0 else "B: ") + s)
    return out

def get_speaker(line: str) -> str:
    line = line.strip()
    if line.startswith("A:"): return "A"
    if line.startswith("B:"): return "B"
    return "U"

def build_input_for_infer(utterances):
    assert len(utterances) >= 2, "최소 2줄 이상 필요"
    utts = ensure_speaker_prefixes(utterances)
    k = len(utts) - 1
    ctx_lines = utts[:k]
    tgt_line  = utts[k]
    tgt_spk   = get_speaker(tgt_line)
    parts = []
    if ctx_lines:
        parts.append("[CTX]")
        parts.extend(ctx_lines)
        parts.append("[/CTX]")
    parts.append(f"[TGT_SPK={tgt_spk}] {tgt_line} [/TGT]")
    return "\n".join(parts)

# (선택) 응급 가드 – 고백/연애 맥락의 "~줄래?" 과탐 완화
LOVE_SAFE = re.compile(
    r"(사랑|좋아|결혼|연애|고백|프로포즈).*(줄래|주겠니|줄\s*수\s*있어)\??$",
    re.IGNORECASE
)
def apply_guards(formatted_text: str, pred_label: str, prob_toxic: float, threshold: float) -> str:
    if pred_label == "toxic" and LOVE_SAFE.search(formatted_text.replace("\n", " ")):
        if prob_toxic < max(0.9, threshold + 0.3):
            return "non_toxic"
    return pred_label

# 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_DIR, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(BASE_DIR, trust_remote_code=True).to(DEVICE).eval()
TH = load_threshold(BASE_DIR, default=0.5)

@torch.inference_mode()
def predict_toxic_with_context(utterances, threshold=None, use_guard=True):
    """
    utterances: ["A: ...", "B: ...", "A: ...", "B: ..."] (길이 2 이상 자유)
    threshold: None이면 저장된 best threshold 사용
    """
    if threshold is None:
        threshold = TH

    formatted = build_input_for_infer(utterances)
    inputs = tokenizer(formatted, truncation=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=-1).squeeze(0).tolist()
    prob_non_toxic, prob_toxic = float(probs[0]), float(probs[1])

    pred_id = int(prob_toxic >= threshold)
    pred_label = {0:"non_toxic", 1:"toxic"}[pred_id]

    if use_guard:
        pred_label = apply_guards(formatted, pred_label, prob_toxic, threshold)

    return {
        "use_context": "ALL",
        "text": formatted,
        "prob_toxic": prob_toxic,
        "threshold": threshold,
        "pred_label": pred_label
    }

# ---- 콘솔 입력 예시 ----
if __name__ == "__main__":
    print("A→B→A→B 순서로 최대 4줄(혹은 그 이상) 입력하세요. (빈 줄로 종료)")
    lines = []
    for i in range(8):  # 원하면 더 길게 입력
        s = input(f"{i+1}번째 줄: ").strip()
        if not s: break
        lines.append(s)

    if len(lines) < 2:
        raise ValueError("최소 2줄 이상 입력해야 합니다.")

    out = predict_toxic_with_context(lines)
    print("\n=== 결과 ===")
    print("입력(모델에 투입된 실제 문자열):")
    print(out["text"])
    print("\n예측:")
    print({"prob_toxic": out["prob_toxic"], "threshold": out["threshold"], "pred_label": out["pred_label"]})


A→B→A→B 순서로 최대 4줄(혹은 그 이상) 입력하세요. (빈 줄로 종료)
1번째 줄: A: 오늘 비 오니까 국물 있는거 먹자
2번째 줄: B: 칼국수 어때?
3번째 줄: A: 좋지!
4번째 줄: B: 넌 면보다 질척거림 ㅋㅋ
5번째 줄: 

=== 결과 ===
입력(모델에 투입된 실제 문자열):
[CTX]
A: 오늘 비 오니까 국물 있는거 먹자
B: 칼국수 어때?
A: 좋지!
[/CTX]
[TGT_SPK=B] B: 넌 면보다 질척거림 ㅋㅋ [/TGT]

예측:
{'prob_toxic': 0.012123801745474339, 'threshold': 0.44999999999999996, 'pred_label': 'non_toxic'}


In [ ]:
import json

data_new = []
with open("/content/drive/MyDrive/metrics/추가데이터 150개#1.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:  # 빈 줄 건너뛰기
            continue
        try:
            data_new.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("❌ JSONDecodeError 발생 줄:", line)
            raise e

print(len(data_new), "개 로드 완료")

150 개 로드 완료


In [ ]:
NEW_DATA_PATH = "/content/drive/MyDrive/metrics/추가데이터 150개#1.jsonl"
BASE_DIR = "/content/drive/MyDrive/model_backup/results_ctx_tgt_best"
# ===== 2. JSONL 로드 =====
data_new = load_jsonl(NEW_DATA_PATH)
print(f"📂 추가 데이터 로드: {len(data_new)} rows")

# ===== 3. 기존 전처리 재사용 =====
records_new = []
bad = 0
for r in data_new:
    utts = r.get("utterances") or r.get("context") or []
    idx = r.get("target_index", None)
    lbl = r.get("label", None)
    if not isinstance(utts, list) or idx is None or idx < 0 or idx >= len(utts):
        bad += 1
        continue
    records_new.append({
        "dialogue_id": r.get("dialogue_id", r.get("id", "")),
        "text": build_input(utts, idx),
        "label": map_label(lbl)
    })
if bad:
    print(f"⚠️ 무시된 레코드: {bad}")

# ===== 4. Dataset 변환 =====
df_new = pd.DataFrame(records_new)
print("라벨 분포:\n", df_new["label"].value_counts())

new_ds = HFDataset.from_pandas(df_new.rename(columns={"label": "labels"}))

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

new_ds = new_ds.map(tokenize, batched=True)
keep_cols = ["input_ids", "attention_mask", "labels"]
new_ds.set_format(type="torch", columns=keep_cols)

# ===== 5. 기존 모델 로드 =====
model = AutoModelForSequenceClassification.from_pretrained(BASE_DIR, trust_remote_code=True)

# ===== 5-1. KoBERT 토크나이저 저장 버그 패치 =====
orig_save_vocab = getattr(tokenizer, "save_vocabulary", None)
if callable(orig_save_vocab):
    def _patched_save_vocabulary(save_directory, *args, **kwargs):
        if "filename_prefix" in kwargs:
            kwargs.pop("filename_prefix")
        return orig_save_vocab(save_directory, *args, **kwargs)
    tokenizer.save_vocabulary = _patched_save_vocabulary

# ===== 6. 이어서 학습 =====
continue_args = TrainingArguments(
    output_dir=BASE_DIR,
    logging_dir=os.path.join(BASE_DIR, "logs_continue"),
    eval_strategy="epoch",     # eval_strategy → evaluation_strategy
    logging_strategy="steps",        # step 단위로 로그
    logging_steps=10,                 # 10 step마다 로그 출력
    disable_tqdm=False,               # tqdm 진행바 표시
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    num_train_epochs=2,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

continue_trainer = Trainer(
    model=model,
    args=continue_args,
    train_dataset=new_ds,
    eval_dataset=val_ds,  # 기존 검증 데이터
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("🚀 추가 데이터로 이어서 학습 시작")
continue_trainer.train()
print("✅ 추가 학습 완료")

# ===== 7. 모델 저장 =====
continue_trainer.save_model(BASE_DIR)
tokenizer.save_pretrained(BASE_DIR)
print("💾 모델과 토크나이저 저장 완료:", BASE_DIR)

📂 추가 데이터 로드: 150 rows
라벨 분포:
 label
0    150
Name: count, dtype: int64


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/tmp/ipython-input-2793238249.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  continue_trainer = Trainer(


🚀 추가 데이터로 이어서 학습 시작


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.273200,1.987557,0.524811,1.000000,0.003446,0.006868,0.847923
2,0.011300,2.213041,0.523168,0.000000,0.000000,0.000000,0.895848


✅ 추가 학습 완료
💾 모델과 토크나이저 저장 완료: /content/drive/MyDrive/model_backup/results_ctx_tgt_best
